# Chapter 19. Concurrency Models in Python

## The Big Picture

Q. Starting threads and processes is easy enough, but how do you keep track of them?

You don't automatically know when it's done, and getting back results or errors requires setting up some communication channel.

A courtine is cheap to start. If you start a coroutine using the `await` keyword, it's easy to get a value returned by it, it can be safely cancelled and you have a clear stie to catch exception. But coroutines are often started by async framework and that can make them as hard to monitor as threads or processes.

## A Bit of Jargon

* Concurrency
 - The ability to handle multiple pending tasks, making progress one at a time or in parallel so that each of them eventually succeeds or fails.

* Parallelism
 - The ability to execute multiple computations at the same time. This requires a multicore CPU, multiple CPUs, a GPU or multiple computers in a cluster.

* Execution unit
 - General term for objs that execute code concurrently, each with independent state and call stack. (Python natively supports 3 kinds of execution units: processes, threads and coroutines)

* Process
 - instance of a comp program while it is running, using memory and a slice of CPU time.

* Thread
 - An execution unit within a single process. When a process satrts, it uses a single thread: the main thread. A process can create more threads to operate concurrently by calling operating system APIs. Threads within a process share the same memory space, which holds live Python objects. This allows easy data sharing between threads, but can also lead to corrupted data when more than one thread updates the same obj concurrently.

* Coroutine
 - A fcn that can suspend itself and resume later. In Python, *classic coroutines* are built from generator fcn, and *native coroutines* are defined with `async def`.



## Processes, Threads, and Python's Infamous GIL

1. Each instance of Python interpreter is a process. You can start additional Python processes using `multiprocessing` or `concurrent.futures` library. Python's `subprocess` lib is designed to launch processes to run external programs, regardless of the languages used to write them.

2. The Python interpreter uses a single thread to run the user's program and the memory garbage collector. You can start additional Python threads using the `threading` or `concurrent.futures` lib.

3. Access to object ref counts and other internal interpreter state is controlled by a lock, the Global Interpreter Lock (GIL). Only one python thread can hold the GIL at any time.

4. To prevent a Python thread from holding the GIL indefinitely, Python bytecode interpreter pauses the current Python thread every 5ms by default.

5. When we write Python code, we have no control over the GIL. but a built-in fcn or an extension written in C (or any lang that interfaces at the Python/C api level) can release the GIL while running time-consuming tasks.

6. Every Python standard lib fcn that makes a `syscall` releases the GIL. This includes all fcns that perform disk I/O, network I/O, and `time.sleep()`. Many CPU intensive fcns in the NumPy/SciPy libraries also release the GIL.

7. Extensions that integrate at the Python/C API level can also launch other non-Python threads that are not affected by the GIL. Such GIL-free threads generally cannot change Python objs, but they can read from and write to the memory underlying objects that support the "buffer protocol" such as `bytearray`, `array.array` and Numpy arrays.

8. The effect of the GIL on network programming with Python threads is relatively small, because I/O fcn release the GIL, and r/w to the network always implies high latency. Consequently, each individual thread spends a lot of time waiting anyway, so their execution can be interleaved w/o major impact on the overall throughput.

9. Contention over the GIL slows down compute-intensive Python threads. Sequential, single-threaded code is simpler and faster for such tasks.

10. To run CPU-intensive Python code on multiple cores, you must use multiple Python processes.

## A Concurrent Hello World

### Spinner with Threads

Start a fcn that blocs for 3 seconds while animating characters in the terminal to let the user know that the program is "thinking" and not stalled.

In [ ]:
# spinner_thread.py

import itertools
import time
from threading import Thread, Event

# This fcn will run in a separate thread
# The done arg is an instance of threading.Event
# a symple obj to synchronize threads
def spin(msg: str, done: Event) -> None:
  for char in itertools.cycle(r'\|/-'): # inf loop
    status = f'\r{char} {msg}' # trick for text-mode anim
    # flush=True forcibly flushes the output stream
    # independent of what default data buffering the file stream has
    print(status, end='', flush=True)
    if done.wait(.1): # Event.wait(timeout=None) method returns True
    # when the event is set by another thread
      break

    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
  time.sleep(3) # calling sleep blocks the main thread
                # but GIL is released so the spinner thread can proceed
  return 42

# threading.Event class is Python's simplest signalling mechanism
# to coordinate threads
# supervisior() returns the result of slow
def supervisor() -> int:
  done = Event()
  spinner = Thread(target=spin, args=('thinking!', done))
  print(f'spinner object: {spinner}')
  spinner.start() # start the thread
  result = slow() # call slow, which blocks the main thread.
                  # Meanwhile, the secondary thread is running the spinner anim
  done.set() # set the Event flag to True
  spinner.join() # wait until the spinner thread finishes
  return result

def main() -> None:
  result = supervisor()
  print(f" Answer: {result}")

if __name__ == '__main__':
  main()

spinner object: <Thread(Thread-10 (spin), initial)>
| thinking! Answer: 42


In [ ]:
!python spinner_thread.py

spinner object: <Thread(Thread-1 (spin), initial)>
| thinking! Answer: 42


### Spinner with Processes

The `multiprocessing` package supports running concurrent tasks in separate Python processes instead of threads. When you create a `multiprocessing.Process` instance, a whole new Python interpreter is started as a child process in the background.

In [ ]:
# spinner_proc.py
import itertools
import time
from multiprocessing import Process, Event
from multiprocessing import synchronize # Mypy forces us to import syncrhonize

# This fcn will run in a separate thread
# The done arg is an instance of threading.Event
# a symple obj to synchronize threads
def spin(msg: str, done: synchronize.Event) -> None:
  for char in itertools.cycle(r'\|/-'): # inf loop
    status = f'\r{char} {msg}' # trick for text-mode anim
    # flush=True forcibly flushes the output stream
    # independent of what default data buffering the file stream has
    print(status, end='', flush=True)
    if done.wait(.1): # Event.wait(timeout=None) method returns True
    # when the event is set by another thread
      break

    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
  time.sleep(3) # calling sleep blocks the main thread
                # but GIL is released so the spinner thread can proceed
  return 42

def supervisor() -> int:
  done = Event()
  spinner = Process(target=spin, args=('thinking!', done))
  print(f"spinner object: {spinner}")
  spinner.start()
  result = slow()
  done.set()
  spinner.join()
  return result

def main() -> None:
  result = supervisor()
  print(f" Answer: {result}")

if __name__ == '__main__':
  main()

In [ ]:
!python spinner_proc.py

spinner object: <Process name='Process-1' parent=13302 initial>
| thinking! Answer: 42


The basic API of `threading` and `multiprocessing` are simlar but their implementation is very different, and `multiprocessing` has a much larger API to handle the added complexity of multiprocess programming.

18:57
### Spinner with Coroutines

Coroutines are driven by an app-level event loop that manages a queue of pending coroutines, drives them one by one, motitors events triggered by I/O operations initiated by coroutines and passes control back to the corresponding coroutine when each event happens.

In [ ]:
# spinner_async.py
import itertools
import time
from multiprocessing import Process, Event
from multiprocessing import synchronize # Mypy forces us to import syncrhonize
import asyncio

async def spin(msg: str) -> None:
  for char in itertools.cycle(r'\|/-'): # inf loop
    status = f'\r{char} {msg}' # trick for text-mode anim
    # flush=True forcibly flushes the output stream
    # independent of what default data buffering the file stream has
    print(status, end='', flush=True)
    try:
      await asyncio.sleep(.1)
    except asyncio.CancelledError:
      break

    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

async def slow() -> int:
  # await asyncio.sleep(3)
  return 42

async def supervisor() -> int:
  # schedules the eventual execution of spin
  # immediately returning an instance of asyncio.Task
  spinner = asyncio.create_task(spin("thinking!"))
  print(f"spinner object: {spinner}")
  # blocking supervisor() until slow returns
  result = await slow()
  # task.cancel method raises a Cancelled Error Exception
  # inside the spin coroutine
  spinner.cancel()
  return result

def main() -> None: # only regular fcn defined in this program
  # asyncio.run fcn starts the event loop to drive the coroutine that will
  # eventually set the other coroutines in motion.
  # main fcn will stack blocked until supervision returns
  # The return value of `supervisor` will be the return value
  # of asyncio.run
  result = asyncio.run(supervisor())
  print(f" Answer: {result}")

if __name__ == '__main__':
  main()

Three main ways of running a coroutine:
 - `asyncio.run(coro())`: Called from a regular fcn to drive a coroutine object that usually is the entry point for all the async code in the program.
 - `asyncio.create_task(coro())`: Called from a coroutine to schedule another coroutine to execute eventually. It returns a `Task` instance, an object that wraps the coroutine object and provides methods to control and query its state.
 - `await coro()`: Called from a coroutine to transfer control to the coroutine object returned by `coro()`. This suspends the current coroutine until the body of `coro` returns.

In [6]:
!python spinner_async.py

spinner object: <Task pending name='Task-2' coro=<spin() running at /content/spinner_async.py:8>>
| thinking! Answer: 42


## The Real Impact of the GIL

In [4]:
# primes.py
import math

def is_prime(n: int) -> bool:
  if n < 2:
    return False
  if n == 2:
    return True
  if n % 2 == 0:
    return False

  root = math.isqrt(n)
  for i in range(3, root + 1, 2):
    if n % i == 0:
      return False
  return True

In [5]:
is_prime(5_000_111_000_222_021)

True

### Quick Quiz

What would happen to the spinner animation if you made the following changes, assuming that `n = 5_000_111_000_222_021`

1. In spinner_proc.py, replace `time.sleep(3)` with a call to `is_prime(n)`?

My answer. As the spinner runs in a seperate process, the program will run as expected.

In [6]:
!python spinner_proc.py

spinner object: <Process name='Process-1' parent=2138 initial>
| thinking! Answer: True


2. In spinner_thread.py, replace `time.sleep(3)` with a call to `is_prime(n)`?

My answer. Maybe spinner won't work because our `is_prime` does not implement the logic for releasing GIL lock?

-> Wrong: Why? the spinner keeps spinning because Python suspends the running thread every 5ms, making the GIL available to other pending threads. Therefore, the main thread running `is_prime` is interrupted every 5ms, allowing the secondary thread to woke up and iterate once through the for loop, until it calls the `wait` method of the `done` event, at which time it will release the GIL.

We got away with a compute-intensive task using threading in this simple experiment because there are only two threads: one hogging the CPU, and the other waking up only 10 times per seconds to update the spinner.

But if you have two or more threads vying for a lot of CPU times, your program will be slower than sequential code.

In [7]:
!python spinner_thread.py # Oh it works!

spinner object: <Thread(Thread-1 (spin), initial)>
\ thinking! Answer: True


3. In spinner_async.py, replace await `asyncio.sleep(3)` with a call to `is_prime(n)`?

My answer. As `is_prime` blocks the main thread, the spinner will not work.

In [10]:
!python spinner_async.py

spinner object: <Task pending name='Task-2' coro=<spin() running at /content/spinner_async.py:26>>
/ thinking! Answer: True


#### Power Napping with sleep(0)
One way to keep the spinner alive is to rewrite `is_prime` as a coroutine and periodically call `asyncio.sleep(0)` in an `await` expression to yield control back to the event loop.

However, be aware this will slow down `is_prime` and will slow down the event loop and your whole program with it.

Using `await asyncio.sleep(0)` should be considered a stopgap measure before we refactor our async code to delegate CPU-intensive computations to another process.

## A Homegrown Process Pool



In [ ]:
#!/usr/bin/env python3

# sequential.py

from time import perf_counter